In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import folium
from folium.plugins import MarkerCluster
import random # <--- Importamos el generador de aleatoriedad

# --- CONFIGURACIÓN DEL PROYECTO ---
tema = "accidente vial policia"
sitio = "yucatan.com.mx" # Usaremos el Diario de Yucatán para variar, o puedes volver a eluniversal
ubicacion_filtro = "Yucatan"

# 1. EXTRACCIÓN (Google News RSS)
url = f"https://news.google.com/rss/search?q={tema}+{ubicacion_filtro}+site:{sitio}&hl=es-419&gl=MX&ceid=MX:es-419"
print(f"📡 Buscando noticias sobre '{tema}' en {sitio}...")

respuesta = requests.get(url)
soup = BeautifulSoup(respuesta.content, 'xml')
items = soup.find_all('item')

data = []
for item in items:
    data.append({
        'Titulo': item.title.text,
        'Fecha': item.pubDate.text,
        'Link': item.link.text
    })

df = pd.DataFrame(data)
print(f"✅ Se encontraron {len(df)} noticias recientes.")

# --- GEOCODIFICACIÓN ---
geolocator = Nominatim(user_agent="portafolio_william_v2")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# Lista de municipios clave para buscar en los títulos
municipios = [
    "Mérida", "Kanasín", "Valladolid", "Tizimín", "Progreso", "Umán", 
    "Tekax", "Ticul", "Chemax", "Motul", "Hunucmá", "Oxkutzcab", "Izamal", 
    "Celestún", "Sisal", "Petad", "Thó", "Caucel", "Altabrisa", "Centro"
]

def obtener_coordenadas(titulo):
    for municipio in municipios:
        if municipio.lower() in titulo.lower():
            try:
                # Buscamos la coordenada del municipio
                location = geolocator.geocode(f"{municipio}, Yucatan, Mexico")
                if location:
                    # --- AQUÍ ESTÁ EL TRUCO DEL JITTERING ---
                    # Agregamos un ruido aleatorio entre -0.005 y 0.005 grados (aprox 500 metros)
                    lat_noise = random.uniform(-0.005, 0.005)
                    lon_noise = random.uniform(-0.005, 0.005)
                    
                    return location.latitude + lat_noise, location.longitude + lon_noise, municipio
            except:
                return None, None, None
    return None, None, None

print("📍 Iniciando geocodificación inteligente (con Jittering)...")
# Aplicamos la función a cada fila (esto puede tardar unos segundos)
df[['Lat', 'Lon', 'Lugar']] = df['Titulo'].apply(lambda x: pd.Series(obtener_coordenadas(x)))

# Eliminamos las noticias que no pudimos geolocalizar
df_clean = df.dropna(subset=['Lat'])
print(f"🗺️  Noticias geolocalizadas exitosamente: {len(df_clean)}")

# --- MAPEO INTERACTIVO ---
mapa = folium.Map(location=[20.967, -89.623], zoom_start=10, tiles="CartoDB positron")

# Usamos MarkerCluster: Agrupa los puntos cuando te alejas y los separa cuando te acercas
marker_cluster = MarkerCluster().add_to(mapa)

for idx, row in df_clean.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Lon']],
        popup=f"<b>{row['Lugar']}</b><br>{row['Titulo']}<br><a href='{row['Link']}' target='_blank'>Ver nota</a>",
        icon=folium.Icon(color="red", icon="exclamation-sign")
    ).add_to(marker_cluster)

# Guardar y mostrar
nombre_mapa = "mapa_seguridad_yucatan.html"
mapa.save(nombre_mapa)
print(f"🚀 ¡Listo! Abre '{nombre_mapa}' para ver el resultado disperso.")
mapa

📡 Buscando noticias sobre 'accidente vial policia' en yucatan.com.mx...
✅ Se encontraron 100 noticias recientes.
📍 Iniciando geocodificación inteligente (con Jittering)...
🗺️  Noticias geolocalizadas exitosamente: 56
🚀 ¡Listo! Abre 'mapa_seguridad_yucatan.html' para ver el resultado disperso.
